# PDF

[Portable Document Format (PDF)](https://en.wikipedia.org/wiki/PDF), ISO 32000으로 표준화된 파일 형식은 Adobe가 1992년에 문서를 제시하기 위해 개발했으며, 이는 응용 소프트웨어, 하드웨어 및 운영 시스템에 독립적인 방식으로 텍스트 서식 및 이미지를 포함합니다.

이 가이드는 `PDF` 문서를 LangChain [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document) 형식으로 로드하는 방법을 다룹니다. 이 형식은 다운스트림에서 사용됩니다.

LangChain은 다양한 PDF 파서와 통합됩니다. 일부는 간단하고 상대적으로 저수준이며, 다른 일부는 OCR 및 이미지 처리를 지원하거나 고급 문서 레이아웃 분석을 수행합니다. 

올바른 선택은 사용자의 애플리케이션에 따라 달라집니다.

**참고**

- [LangChain 도큐먼트](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/)

## AutoRAG 팀에서의 PDF 실험

AutoRAG 에서 진행한 실험을 토대로 작성한 순위표

아래 표기된 숫자는 등수를 나타냅니다. (The lower, the better)

| | PDFMiner | PDFPlumber | PyPDFium2 | PyMuPDF | PyPDF2 |
|----------|:---------:|:----------:|:---------:|:-------:|:-----:|
| Medical  | 1         | 2          | 3         | 4       | 5     |
| Law      | 3         | 1          | 1         | 3       | 5     |
| Finance  | 1         | 2          | 2         | 4       | 5     |
| Public   | 1         | 1          | 1         | 4       | 5     |
| Sum      | 5         | 5          | 7         | 15      | 20    |

출처: [AutoRAG Medium 블로그](https://velog.io/@autorag/PDF-%ED%95%9C%EA%B8%80-%ED%85%8D%EC%8A%A4%ED%8A%B8-%EC%B6%94%EC%B6%9C-%EC%8B%A4%ED%97%98#%EC%B4%9D%ED%8F%89)

In [6]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

**참고**: 위의 파일은 `data` 폴더 내에 다운로드 받으세요

In [5]:
def show_metadata(docs):
    if docs:
        print("[metadata]")
        print(list(docs[0].metadata.keys()))
        print("\n[examples]")
        max_key_length = max(len(k) for k in docs[0].metadata.keys())
        for k, v in docs[0].metadata.items():
            print(f"{k:<{max_key_length}} : {v}")

## PyPDF

여기에서는 `pypdf`를 사용하여 PDF를 문서 배열로 로드하며, 각 문서는 `page` 번호와 함께 페이지 내용 및 메타데이터를 포함합니다.

In [8]:
# 설치
# !pip install -qU pypdf

In [9]:
# FILE_PATH = "./data/SPRI_AI_Brief_2023년12월호_F.pdf"
FILE_PATH = "./data/2103_page14.pdf"

In [10]:
from langchain_community.document_loaders import PyPDFLoader

# 파일 경로 설정
loader = PyPDFLoader(FILE_PATH)

# PDF 로더 초기화
docs = loader.load()

# 문서의 내용 출력
# print(docs[10].page_content[:300])
print(docs[0].page_content[:])

14 Z. Shen et al. 
6 Conclusion 
LayoutParser provides a comprehensive toolkit for deep learning-based document 
image analysis. The off-the-shelf library is easy to install, and can be used to 
build flexible and accurate pipelines for processing documen ts with complicated 
structures. It also supports high-level customization and enables easy labeling and 
training of DL models on unique documen t image datasets. The LayoutParser 
community platform facilitates sharing DL models and DIA pipelines, inviting 
discussion and promoting code reproducibility and reusability. The LayoutParser 
team is committed to keeping the library updated continuously and bringing 
the most recent advances in DL-based DIA, such as multi-modal documen t 
modeling [37, 36, 9] (an upcoming priority), to a diverse audience of end-users. 
Acknowled gements We thank the anonymous reviewers for their comments 
and suggestions. This project is supported in part by NSF Grant OIA-2033558 
and funding from the Har

In [11]:
# 메타데이터 출력
show_metadata(docs)

[metadata]
['producer', 'creator', 'creationdate', 'author', 'moddate', 'title', 'source', 'total_pages', 'page', 'page_label']

[examples]
producer     : Adobe Acrobat (64-bit) 25 Paper Capture Plug-in
creator      : PyPDF
creationdate : 2025-08-22T12:40:57+09:00
author       : Heejin Park
moddate      : 2025-08-22T12:48:05+09:00
title        : C:\Users\park0\Downloads\2103.15348v2.pdf
source       : ./data/2103_page14.pdf
total_pages  : 1
page         : 0
page_label   : 1


### PyPDF(OCR)

일부 PDF에는 스캔된 문서나 그림 내에 텍스트 이미지가 포함되어 있습니다. `rapidocr-onnxruntime` 패키지를 사용하여 이미지에서 텍스트를 추출할 수도 있습니다.

In [12]:
# 설치
# !pip install -qU rapidocr-onnxruntime

In [13]:
FILE_PATH = "./data/2103_page4.pdf"

In [14]:
# PDF 로더 초기화, 이미지 추출 옵션 활성화
# loader = PyPDFLoader("https://arxiv.org/pdf/2103.15348.pdf", extract_images=True)
loader = PyPDFLoader(FILE_PATH, extract_images=True)
# PDF 페이지 로드
docs = loader.load()

# 페이지 내용 접근
print(docs[0].page_content[:])

4 Z. Shen et al. 
.z-
Efficient Data Annotation I f 
Customized Model Training I ~ I Layout Detection Models I ~ I DIA Pipeline Sharing ’ 
'—` 
OCRModule 〕一Layout Data Structure ’ ’ 
一 Fig.1: The overall architecture of LayoutParser. For an input document image, 
the core LayoutParser library provides a set of off-the-shelf tools for layout 
detection, OCR, visualization, and storage, backed by a carefully designed layout 
data structure. LayoutParser also supports high level customization via efficient 
layout annotation and model training functions. These improve model accuracy 
on the target samples. The community platform enables the easy sharing of DIA 
models and whole digitization pipelines to promote reusability and reproducibility. 
A collection of detailed documen tation, tutorials and exemplar projects make 
LayoutParser easy to learn and use. 
AllenNLP [8] and transformers [34] have provided the community with complete 
DL-based support for developing and deploying models f

In [15]:
show_metadata(docs)

[metadata]
['producer', 'creator', 'creationdate', 'author', 'moddate', 'title', 'source', 'total_pages', 'page', 'page_label']

[examples]
producer     : Adobe Acrobat (64-bit) 25 Paper Capture Plug-in
creator      : PyPDF
creationdate : 2025-08-22T12:39:06+09:00
author       : Heejin Park
moddate      : 2025-08-22T12:47:22+09:00
title        : C:\Users\park0\Downloads\2103.15348v2.pdf
source       : ./data/2103_page4.pdf
total_pages  : 1
page         : 0
page_label   : 1


## PyMuPDF

**PyMuPDF** 는 속도 최적화가 되어 있으며, PDF 및 해당 페이지에 대한 자세한 메타데이터를 포함하고 있습니다. 페이지 당 하나의 문서를 반환합니다:

In [16]:
# 설치
# !pip install -qU pymupdf

In [17]:
from langchain_community.document_loaders import PyMuPDFLoader

# PyMuPDF 로더 인스턴스 생성
loader = PyMuPDFLoader(FILE_PATH)

# 문서 로드
docs = loader.load()

# 문서의 내용 출력
print(docs[0].page_content[:])

4 
Z. Shen et al. 
.z-
Efficient Data Annotation I 
f 
Customized Model Training I ~ 
I Layout Detection Models I ~ 
I 
DIA Pipeline Sharing 
’ 
'—` 
OCRModule 
〕一
Layout Data Structure 
’ 
’ 
一
Fig.1: The overall architecture of Layout Parser. For an input document image, 
t he core Layout Parser library provides a set of off-the-shelf tools for layout 
detection, OCR, visualization, and storage, backed by a carefully designed layout 
data struct ure. Layout Parser also supports high level customization via efficient 
layout annotation and model training functions. These improve model accuracy 
on the target samples. The community platform enables t he easy sharing of DIA 
models and whole digitization pipelines to promote reusability and reproducibility. 
A collection of det ailed documentation, t utorials and exemplar projects make 
Layout Parser easy to learn and use. 
AllenNLP [8] and transformers [34] have provided the community with complete 
DL-based support for developing and 

In [18]:
show_metadata(docs)

[metadata]
['producer', 'creator', 'creationdate', 'source', 'file_path', 'total_pages', 'format', 'title', 'author', 'subject', 'keywords', 'moddate', 'trapped', 'modDate', 'creationDate', 'page']

[examples]
producer     : Adobe Acrobat (64-bit) 25 Paper Capture Plug-in
creator      : 
creationdate : 2025-08-22T12:39:06+09:00
source       : ./data/2103_page4.pdf
file_path    : ./data/2103_page4.pdf
total_pages  : 1
format       : PDF 1.7
title        : C:\Users\park0\Downloads\2103.15348v2.pdf
author       : Heejin Park
subject      : 
keywords     : 
moddate      : 2025-08-22T12:47:22+09:00
trapped      : 
modDate      : D:20250822124722+09'00'
creationDate : D:20250822123906+09'00'
page         : 0


## Unstructured

[Unstructured](https://unstructured-io.github.io/unstructured/)는 Markdown이나 PDF와 같은 비구조화된 또는 반구조화된 파일 형식을 다루기 위한 공통 인터페이스를 지원합니다. 

LangChain의 [UnstructuredPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.UnstructuredPDFLoader.html)는 Unstructured와 통합되어 PDF 문서를 LangChain [Document](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html) 객체로 파싱합니다.

In [1]:
# 설치
# !pip install -qU unstructured
FILE_PATH = "./data/SPRI_AI_Brief_2023년12월호_F.pdf"

In [3]:
from langchain_community.document_loaders import UnstructuredPDFLoader

# UnstructuredPDFLoader 인스턴스 생성
loader = UnstructuredPDFLoader(FILE_PATH)

# 데이터 로드
docs = loader.load()

# 문서의 내용 출력
print(docs[0].page_content[:300])

2023년 12월호

2023년 12월호

Ⅰ. 인공지능 산업 동향 브리프

1. 정책/법제

▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 ························· 1

▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의 ··························· 2

▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언 ··························· 3

▹ 미국 법원, 예술가들이 생성 AI 기업에 제


In [6]:
show_metadata(docs)

[metadata]
['source']

[examples]
source : ./data/SPRI_AI_Brief_2023년12월호_F.pdf


내부적으로 비정형에서는 텍스트 청크마다 서로 다른 "**요소**"를 만듭니다. 기본적으로 이들은 결합되어 있지만 `mode="elements"`를 지정하여 쉽게 분리할 수 있습니다.

In [7]:
# UnstructuredPDFLoader 인스턴스 생성(mode="elements")
loader = UnstructuredPDFLoader(FILE_PATH, mode="elements")

# 데이터 로드
docs = loader.load()

# 문서의 내용 출력
print(docs[0].page_content)

2023년 12월호


이 특정 문서에 대한 전체 요소 유형 집합을 참조하세요

In [8]:
set(doc.metadata["category"] for doc in docs)  # 데이터 카테고리 추출

{'ListItem', 'NarrativeText', 'Title', 'UncategorizedText'}

In [ ]:
show_metadata(docs)

## PyPDFium2

In [21]:
from langchain_community.document_loaders import PyPDFium2Loader

# PyPDFium2 로더 인스턴스 생성
loader = PyPDFium2Loader(FILE_PATH)

# 데이터 로드
docs = loader.load()

# 문서의 내용 출력
print(docs[10].page_content[:300])

SPRi AI Brief | 
2023-12월호
8
코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개
n 코히어와 12개 기관이 광범위한 데이터셋에 대한 감사를 통해 원본 데이터 출처, 재라이선스 상태, 작성자 등 다양한 정보를 제공하는 ‘데이터 출처 탐색기’ 플랫폼을 출시
n 대화형 플랫폼을 통해 개발자는 데이터셋의 라이선스 상태를 쉽게 파악할 수 있으며 데이터셋의 
구성과 계보도 추적 가능
KEY Contents
£ 데이터 출처 탐색기, 광범위한 데이터셋 정보 제공을 통해 데이터 투명성 향상
n AI 기업 코히어(Co


c:\Users\park0\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-hTuzgesB-py3.11\Lib\site-packages\pypdfium2\_helpers\textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [ ]:
show_metadata(docs)

## PDFMiner

In [22]:
from langchain_community.document_loaders import PDFMinerLoader

# PDFMiner 로더 인스턴스 생성
loader = PDFMinerLoader(FILE_PATH)

# 데이터 로드
docs = loader.load()

# 문서의 내용 출력
print(docs[0].page_content[:300])

2023년  12월호
2023년  12월호

Ⅰ.  인공지능  산업  동향  브리프

  1.  정책/법제 

      ▹  미국,  안전하고  신뢰할  수  있는  AI  개발과  사용에  관한  행정명령  발표    ························· 1

      ▹  G7,  히로시마  AI  프로세스를  통해  AI  기업  대상  국제  행동강령에  합의 ··························· 2

      ▹  영국  AI  안전성  정상회의에  참가한  28개국,  AI  위험에  공동  


In [ ]:
show_metadata(docs)

**PDFMiner**를 사용하여 HTML 텍스트 생성

이 방법은 출력된 HTML 콘텐츠를 `BeautifulSoup`을 통해 파싱함으로써 글꼴 크기, 페이지 번호, PDF 헤더/푸터 등에 대한 보다 구조화되고 풍부한 정보를 얻을 수 있게 하여 텍스트를 의미론적으로 섹션으로 분할하는 데 도움이 될 수 있습니다.

In [23]:
from langchain_community.document_loaders import PDFMinerPDFasHTMLLoader

# PDFMinerPDFasHTMLLoader 인스턴스 생성
loader = PDFMinerPDFasHTMLLoader(FILE_PATH)

# 문서 로드
docs = loader.load()

# 문서의 내용 출력
print(docs[0].page_content[:300])

<html><head>
<meta http-equiv="Content-Type" content="text/html">
</head><body>
<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:612px; height:858px;"></span>
<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>
<div style="position:absolute; border


In [24]:
show_metadata(docs)

[metadata]
['source']

[examples]
source : ./data/SPRI_AI_Brief_2023년12월호_F.pdf


In [25]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(docs[0].page_content, "html.parser")  # HTML 파서 초기화
content = soup.find_all("div")  # 모든 div 태그 검색

In [26]:
import re

cur_fs = None
cur_text = ""
snippets = []  # 동일한 글꼴 크기의 모든 스니펫 수집
for c in content:
    sp = c.find("span")
    if not sp:
        continue
    st = sp.get("style")
    if not st:
        continue
    fs = re.findall("font-size:(\d+)px", st)
    if not fs:
        continue
    fs = int(fs[0])
    if not cur_fs:
        cur_fs = fs
    if fs == cur_fs:
        cur_text += c.text
    else:
        snippets.append((cur_text, cur_fs))
        cur_fs = fs
        cur_text = c.text
snippets.append((cur_text, cur_fs))
# 중복 스니펫 제거 전략 추가 가능성 (PDF의 헤더/푸터가 여러 페이지에 걸쳐 나타나므로 중복 발견 시 중복 정보로 간주 가능)

In [27]:
from langchain_core.documents import Document

cur_idx = -1
semantic_snippets = []
# 제목 가정: 높은 글꼴 크기
for s in snippets:
    # 새 제목 판별: 현재 스니펫 글꼴 > 이전 제목 글꼴
    if (
        not semantic_snippets
        or s[1] > semantic_snippets[cur_idx].metadata["heading_font"]
    ):
        metadata = {"heading": s[0], "content_font": 0, "heading_font": s[1]}
        metadata.update(docs[0].metadata)
        semantic_snippets.append(Document(page_content="", metadata=metadata))
        cur_idx += 1
        continue

    # 동일 섹션 내용 판별: 현재 스니펫 글꼴 <= 이전 내용 글꼴
    if (
        not semantic_snippets[cur_idx].metadata["content_font"]
        or s[1] <= semantic_snippets[cur_idx].metadata["content_font"]
    ):
        semantic_snippets[cur_idx].page_content += s[0]
        semantic_snippets[cur_idx].metadata["content_font"] = max(
            s[1], semantic_snippets[cur_idx].metadata["content_font"]
        )
        continue

    # 새 섹션 생성 조건: 현재 스니펫 글꼴 > 이전 내용 글꼴, 이전 제목 글꼴 미만
    metadata = {"heading": s[0], "content_font": 0, "heading_font": s[1]}
    metadata.update(docs[0].metadata)
    semantic_snippets.append(Document(page_content="", metadata=metadata))
    cur_idx += 1

print(semantic_snippets[4])

page_content='KEY Contents
n 미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령’에 서명하고 
광범위한  행정  조치를  명시
n 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 
보호  △노동자  지원  △혁신과  경쟁  촉진  △국제협력을  골자로  함
' metadata={'heading': '미국,  안전하고  신뢰할  수  있는  AI  개발과  사용에  관한  행정명령  발표 \n', 'content_font': 12, 'heading_font': 15, 'source': './data/SPRI_AI_Brief_2023년12월호_F.pdf'}


## PyPDF 디렉토리

디렉토리에서 PDF를 로드하세요

In [28]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

# 디렉토리 경로
loader = PyPDFDirectoryLoader("data/")

# 문서 로드
docs = loader.load()

# 문서의 개수 출력
print(len(docs))

33


In [30]:
# 문서의 내용 출력
print(docs[0].page_content[:300])

LayoutParser: A Unified Toolkit for Deep 
Learning Based Documen t Image Analysis 
IZOZ 
unr 
It Zejiang Shen1 詞),Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain 
Lee4, Jacob Carlson3, and Weining Li5 
1 Allen Institute for AI 
shannons©allena i.org 
2 Brown University 
ruochen_zhan g©brown.


In [ ]:
# metadata 출력
print(docs[50].metadata)

## PDFPlumber

PyMuPDF와 마찬가지로, 출력 문서는 PDF와 그 페이지에 대한 자세한 메타데이터를 포함하며, 페이지 당 하나의 문서를 반환합니다.

In [31]:
from langchain_community.document_loaders import PDFPlumberLoader

# PDF 문서 로더 인스턴스 생성
loader = PDFPlumberLoader(FILE_PATH)

# 문서 로딩
docs = loader.load()

# 첫 번째 문서 데이터 접근
print(docs[10].page_content[:300])

SPRi AI Brief |
2023-12월호
코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개
KEY Contents
n 코히어와 12개 기관이 광범위한 데이터셋에 대한 감사를 통해 원본 데이터 출처, 재라이선스 상태,
작성자 등 다양한 정보를 제공하는 ‘데이터 출처 탐색기’ 플랫폼을 출시
n 대화형 플랫폼을 통해 개발자는 데이터셋의 라이선스 상태를 쉽게 파악할 수 있으며 데이터셋의
구성과 계보도 추적 가능
£데이터 출처 탐색기, 광범위한 데이터셋 정보 제공을 통해 데이터 투명성 향상
n AI 기업 코히어(Cohere)


In [ ]:
show_metadata(docs)

In [32]:
# pdfminer 호환성 문제 해결을 위한 패키지 설치
import subprocess
import sys


def install_compatible_packages():
    """호환되는 버전의 패키지들 설치"""
    packages = [
        "pdfminer.six==20220319",  # 호환되는 구 버전
        "unstructured[pdf]==0.10.30",  # 호환되는 unstructured 버전
    ]

    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✓ {package} 설치 완료")
        except subprocess.CalledProcessError as e:
            print(f"✗ {package} 설치 실패: {e}")


# 호환 패키지 설치 실행
install_compatible_packages()
print("\n패키지 설치 완료. 커널을 재시작하고 다시 시도해주세요.")

✓ pdfminer.six==20220319 설치 완료
✓ unstructured[pdf]==0.10.30 설치 완료

패키지 설치 완료. 커널을 재시작하고 다시 시도해주세요.
✓ unstructured[pdf]==0.10.30 설치 완료

패키지 설치 완료. 커널을 재시작하고 다시 시도해주세요.
